In [ ]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.3
tf.Session(config=config)

import keras
from keras.models import *
from keras.layers import *
from keras import optimizers
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.backend import tf as ktf
from keras.callbacks import EarlyStopping
from tqdm import tqdm

import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from utils import *
%matplotlib inline
from jupyterthemes import jtplot

jtplot.style()

### Data pipeline

In [ ]:
%%time
X_train = np.load('data/processed/X_train.npy')
print(X_train.shape)
Y_train = np.load('data/processed/Y_train.npy')
print(Y_train.shape)
X_test = np.load('data/processed/X_test.npy')
print(X_test.shape)

# train_data = np.load('models/bottleneck_features_train.npy')
# validation_data = np.load('models/bottleneck_features_validation.npy')
# test_data = np.load('models/bottleneck_features_test.npy')

In [ ]:
# X_train, X_dev, Y_train, Y_dev = train_test_split(X_train, Y_train, test_size=0.25, random_state=0)

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range = 10,        
    horizontal_flip = True,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range=0.2,    
    zoom_range = 0.2,
    fill_mode='nearest')

dev_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
def aug_data(X_train, Y_train, batch_count):
    X, Y = [], []
    count = 0
    for bx, by in train_datagen.flow(X_train, Y_train, batch_size=64):
        for x, y in zip(bx, by):
            X.append(x)
            Y.append(y)
        count+=1
        print(count, end='\r')
        if count>batch_count:
            break
    X = np.asarray(X)
    Y = np.asarray(Y)
    
    return X, Y

In [ ]:
# X, Y = aug_data(X_train, Y_train, 500)
# X = np.load('data/preprocess/X_aug.npy')
# Y = np.load('data/preprocess/Y_aug.npy')

In [ ]:
def top_model(input_shape):
    input_img = Input(input_shape)
    X = GlobalAveragePooling2D()(input_img)
#     X = Flatten(input_shape=input_shape)(input_img)
    X = Dropout(0.2)(X)   

    X = Dense(1024, activation='relu')(X)
    X = Dropout(0.5)(X)
    
    X = Dense(1024, activation='relu')(X)
    X = Dropout(0.5)(X)
    
    X = Dense(120, activation='softmax')(X)
    
    model = Model(inputs=input_img, outputs=X)
    
    model.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
    
    return model

### VGG 

In [ ]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3), classes=1)

In [ ]:
type(vgg_model)

In [ ]:
vgg_model.ad

In [ ]:
# vgg_train_bf = vgg_model.predict(X_train, verbose=1)

In [ ]:
# vgg_test_bf = vgg_model.predict(X_test, verbose=1)

In [ ]:
# np.save('data/processed/vgg_test_bf.npy', vgg_test_bf)
# np.save('data/processed/vgg_train_bf.npy', vgg_train_bf)

In [ ]:
vgg_train_bf = np.load('data/processed/vgg_train_bf.npy')
vgg_test_bf = np.load('data/processed/vgg_test_bf.npy')

In [ ]:
vggtop_model = top_model(vgg_train_bf.shape[1:])

In [ ]:
vggtop_history = vggtop_model.fit(vgg_train_bf, Y_train, batch_size=100, epochs=30, validation_split=0.2,
                             callbacks=[EarlyStopping(monitor='val_acc', patience=3, verbose=1)])

In [ ]:
plot_training(vggtop_history)

## ResNet

In [ ]:
# base_model = ResNet50(input_tensor=Input((224, 224, 3)), weights='imagenet', include_top=False)

In [ ]:
# train_bf = base_model.predict(X_train, verbose=1)

In [ ]:
# test_bf = base_model.predict(X_test, verbose=1)

In [ ]:
# np.save('data/processed/res_test_bf.npy', test_bf)
# np.save('data/processed/res_train_bf.npy', train_bf)

In [ ]:
res_train_bf = np.load('data/processed/res_train_bf.npy')
res_test_bf = np.load('data/processed/res_test_bf.npy')

In [ ]:
restop_model = top_model(res_train_bf.shape[1:])

In [ ]:
restop_history = restop_model.fit(res_train_bf, Y_train, batch_size=100, epochs=30, validation_split=0.2,
                             callbacks=[EarlyStopping(monitor='val_acc', patience=3, verbose=1)])

In [ ]:
plot_training(restop_history)

## InceptionV3

In [ ]:
# inception_model = InceptionV3(input_tensor=Input((224, 224, 3)), weights='imagenet', include_top=False)

In [ ]:
# inc_train_bf = inception_model.predict(X, verbose=1)
# inc_test_bf = inception_model.predict(X_test, verbose=1)

In [ ]:
# np.save('data/processed/inc_test_bf.npy', inc_test_bf)
# np.save('data/processed/inc_train_bf.npy', inc_train_bf)

In [ ]:
%%time
inc_train_bf = np.load('data/processed/inc_train_bf.npy')
Y = np.load('data/processed/Y_aug.npy')
inc_test_bf = np.load('data/processed/inc_test_bf.npy')

In [ ]:
inctop_model = top_model(inc_train_bf.shape[1:])

In [ ]:
inc_history = inctop_model.fit(inc_train_bf, Y, batch_size=100, epochs=25, validation_split=0.2,
             callbacks=[EarlyStopping(monitor='val_acc', patience=3, verbose=1)])
plot_training(inc_history)

In [ ]:
inctop_model.save_weights('models/weights/inctop1.h5')

In [ ]:
inctop_model.load_weights('models/weights/inctop1.h5')

In [ ]:
inctop_model.optimizer.lr = 0.1

In [ ]:
inc_history = inctop_model.fit(inc_train_bf, Y, batch_size=100, epochs=20, validation_split=0.2,
                               callbacks=[EarlyStopping(monitor='val_acc', patience=3, verbose=1)])
plot_training(inc_history)

In [ ]:
inctop_model.save_weights('models/weights/inctop2.h5')

In [ ]:
inctop_model.load_weights('models/weights/inctop2.h5')

In [ ]:
inctop_model.optimizer.lr = 0.01

In [ ]:
inc_history = inctop_model.fit(inc_train_bf, Y, batch_size=100, epochs=5, validation_split=0.2)
plot_training(inc_history)

In [ ]:
inctop_model.save_weights('models/weights/inctop3.h5')

In [ ]:
inctop_model.optimizer.lr = 0.001

In [ ]:
inc_history = inctop_model.fit(inc_train_bf, Y, batch_size=100, epochs=5, validation_split=0.2)
plot_training(inc_history)

In [ ]:
inctop_model.load_weights('models/weights/inctop4.h5')

In [ ]:
inctop_model.evaluate(inc_train_bf, Y)

## Fine tuning

In [ ]:
def ft_model(base_model, top_model_weights_path):
    
    top = top_model(base_model.output_shape[1:])
    top.load_weights(top_model_weights_path)
#     x = base_model.predict(X_train)
#     print(top.evaluate(x, Y_train))
    ft_model = Model(inputs=base_model.inputs, outputs=top(base_model.output))
    
    ft_model.compile(loss='categorical_crossentropy',
                 optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                 metrics=['accuracy'])
    
    return ft_model
    


In [ ]:
inception_model = InceptionV3(input_tensor=Input((224, 224, 3)), weights='imagenet', include_top=False)
for layer in inception_model.layers[:299]:
    layer.trainable = False

In [ ]:
# inc_train_bf = inception_model.predict(X_train, verbose=1)

In [ ]:
inc_ft_model = ft_model(inception_model, 'models/inctop_model.h5')

In [ ]:
# inc_ft_model.evaluate(X_train, Y_train)

In [ ]:
# inc_ft_model.summary()

In [ ]:
inc_ft_history = inc_ft_model.fit(X_train, Y_train, batch_size=50, epochs=20, validation_split=0.2,
             callbacks=[EarlyStopping(monitor='val_acc', patience=3, verbose=1)])

In [ ]:
plot_training(inc_ft_history)

In [ ]:
inc_ft_model2 = ft_model(inception_model, 'models/inctop_model.h5')

In [ ]:
inc_ft_model2.fit_generator(
    train_generator,
    steps_per_epoch=X_train.shape[0] // batch_size,
    epochs=20,
    verbose=1)

## Prediction

In [ ]:
preds = inctop_model.predict(inc_test_bf, verbose=1, batch_size=16)
df_train = pd.read_csv('labels.csv')
df_test = pd.read_csv('sample_submission.csv')
one_hot = pd.get_dummies(df_train['breed'], sparse = True)

sub = pd.DataFrame(preds)
sub.columns = one_hot.columns.values
sub.insert(0, 'id', df_test['id'])

sub.to_csv('sub.csv', index=False)